In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np
import statsmodels.api as sm

In [ ]:
# Fetch CNYUSD exchange rate data
cnyusd_data = yf.download('CNY=X', start='2010-01-01', end='2024-03-31', interval='3mo')

# Extract the adjusted close prices for the CNYUSD exchange rate
cnyusd_prices = cnyusd_data['Adj Close'].reset_index()

# Rename columns for clarity
cnyusd_prices.columns = ['Date', 'CNYUSD']

# Display the first few rows of the CNYUSD data
cnyusd_prices.head()

[*********************100%***********************]  1 of 1 completed


,Date,CNYUSD
0,2010-01-01,6.8159
1,2010-04-01,6.7720
2,2010-07-01,6.6784
3,2010-10-01,6.6124
4,2011-01-01,6.5486


In [ ]:
# Load Excel profile
file_path = 'G:\\My Drive\\semester 3\\Fin 612\\currency report\\Data612.xlsx'  #change this into data/612data.xlsx
xls = pd.ExcelFile(file_path)

# read 'all data' worksheet
data_sheet = pd.read_excel(xls, sheet_name='All data')

# extract data from Q1 2010 to Q1 2024 
china_data = data_sheet.iloc[1:8, 22:79].reset_index(drop=True)  # row 2 to 8 contain china's data 第3行到第9行为中国的数据
usa_data = data_sheet.iloc[8:15, 22:79].reset_index(drop=True)   # rows 9 to 15 contain USA's data 第10行到第16行为美国的数据

# rename columns to quarterly names from Q1 2010 to Q1 2024
quarter_names = [f"Q{i}" for i in range(1, 58)]
china_data.columns = quarter_names
usa_data.columns = quarter_names

In [ ]:
# Create China's and USA's variables as DataFrames
china_indices = {
    'CPI_CHN': china_data.iloc[4].values,
    'PPI_CHN': china_data.iloc[5].values,
    'UR_CHN': china_data.iloc[3].values,
    'FR_CHN': china_data.iloc[2].values,
    'IP_CHN': china_data.iloc[0].values,
    'INT_CHN': china_data.iloc[1].values,
    'CA_CHN': china_data.iloc[6].values
}

usa_indices = {
    'CPI_USA': usa_data.iloc[4].values,
    'PPI_USA': usa_data.iloc[5].values,
    'UR_USA': usa_data.iloc[3].values,
    'FR_USA': usa_data.iloc[2].values,
    'IP_USA': usa_data.iloc[0].values,
    'INT_USA': usa_data.iloc[1].values,
    'CA_USA': usa_data.iloc[6].values
}

In [ ]:
#Extract the two column we need
cnyusd_prices = pd.DataFrame({
    'Date': cnyusd_prices['Date'],
    'CNYUSD': cnyusd_prices['CNYUSD']
})

The CA of China are most of the time positive(Trade surplus) but contains negative values, while the US CA are all negative(Trade deficit). Therefore when calculating the logarithmic difference, there should be invalid values. So we scaled the CA data of both country to (1e-6, 100000) to avoid error.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 创建 MinMaxScaler 实例，将数据缩放到 [1e-6, 100000] 范围
scaler = MinMaxScaler(feature_range=(1e-6, 100000))

# 对中国和美国的 CA 数据进行标准化
china_ca_scaled = scaler.fit_transform(china_data.iloc[6].values.reshape(-1, 1))
usa_ca_scaled = scaler.fit_transform(usa_data.iloc[6].values.reshape(-1, 1))

# 将标准化后的 CA 数据转换为数组
china_ca_scaled = china_ca_scaled.flatten()
usa_ca_scaled = usa_ca_scaled.flatten()

# Create China's and USA's variables as DataFrames
china_indices['CA_CHN']= china_ca_scaled
usa_indices['CA_USA']= usa_ca_scaled


X = pd.DataFrame({
    'ln(CPI_ratio_CHN)': np.log(np.array(china_indices['CPI_CHN'][1:], dtype=float) / np.array(china_indices['CPI_CHN'][:-1], dtype=float)) - np.log(np.array(usa_indices['CPI_USA'][1:], dtype=float) / np.array(usa_indices['CPI_USA'][:-1], dtype=float)),
    'ln(PPI_ratio_CHN)': np.log(np.array(china_indices['PPI_CHN'][1:], dtype=float) / np.array(china_indices['PPI_CHN'][:-1], dtype=float)) - np.log(np.array(usa_indices['PPI_USA'][1:], dtype=float) / np.array(usa_indices['PPI_USA'][:-1], dtype=float)),
    'ln(UR_ratio_CHN)': np.log(np.array(china_indices['UR_CHN'][1:], dtype=float) / np.array(china_indices['UR_CHN'][:-1], dtype=float)) - np.log(np.array(usa_indices['UR_USA'][1:], dtype=float) / np.array(usa_indices['UR_USA'][:-1], dtype=float)),
    'ln(FR_ratio_CHN)': np.log(np.array(china_indices['FR_CHN'][1:], dtype=float) / np.array(china_indices['FR_CHN'][:-1], dtype=float)) - np.log(np.array(usa_indices['FR_USA'][1:], dtype=float) / np.array(usa_indices['FR_USA'][:-1], dtype=float)),
    'ln(INT_ratio_CHN)': np.log(np.array(china_indices['INT_CHN'][1:], dtype=float) / np.array(china_indices['INT_CHN'][:-1], dtype=float)) - np.log(np.array(usa_indices['INT_USA'][1:], dtype=float) / np.array(usa_indices['INT_USA'][:-1], dtype=float)),
    'ln(IP_ratio_CHN)': np.log(np.array(china_indices['IP_CHN'][1:], dtype=float) / np.array(china_indices['IP_CHN'][:-1], dtype=float)) - np.log(np.array(usa_indices['IP_USA'][1:], dtype=float) / np.array(usa_indices['IP_USA'][:-1], dtype=float)),
    'ln(CA_ratio_CHN)': np.log(np.array(china_indices['CA_CHN'][1:], dtype=float) / np.array(china_indices['CA_CHN'][:-1], dtype=float)) - np.log(np.array(usa_indices['CA_USA'][1:], dtype=float) / np.array(usa_indices['CA_USA'][:-1], dtype=float))

})


In [ ]:
# 构建因变量 Y (人民币兑美元汇率变化)
Y = np.log(cnyusd_prices['CNYUSD'].shift(-1)) - np.log(cnyusd_prices['CNYUSD'])

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# 去除缺失值
X = X.dropna()
Y = Y.dropna()

# 拆分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 加入截距项
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [ ]:
# 使用sklearn的LinearRegression进行交叉验证
model_cv = LinearRegression()

# 因为X_train和Y_train包含了回归的自变量和因变量，因此我们可以直接进行交叉验证
# 使用5折交叉验证
cv_scores = cross_val_score(model_cv, X_train, Y_train, cv=5, scoring='r2')

# 输出交叉验证得分
print("Cross-validation R2 scores:", cv_scores)
print("Mean cross-validation R2 score:", np.mean(cv_scores))

# 拟合模型
model_cv.fit(X_train, Y_train)

# 在测试集上进行预测
Y_pred_cv = model_cv.predict(X_test)

# 输出预测结果
print("Test predictions:", Y_pred_cv)

Cross-validation R2 scores: [-2.6267695  -0.58061166 -4.71178597 -1.62110659 -0.03846329]
Mean cross-validation R2 score: -1.9157474019119554
Test predictions: [ 0.00561419 -0.00212892 -0.00299105  0.00063839 -0.00660669  0.00061505
  0.00033834  0.00824024  0.0227797   0.00845509  0.00260992  0.00062292]


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 计算均方误差 (MSE) 和平均绝对误差 (MAE)
mse = mean_squared_error(Y_test, Y_pred_cv)
mae = mean_absolute_error(Y_test, Y_pred_cv)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

# 输出预测结果与真实值的差异
comparison = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred_cv})
print(comparison)

Mean Squared Error (MSE): 0.0006479889030012718
Mean Absolute Error (MAE): 0.022144025717379678
      Actual  Predicted
0  -0.006462   0.005614
5  -0.011751  -0.002129
33  0.035660  -0.002991
13 -0.002893   0.000638
19  0.000726  -0.006607
50 -0.032325   0.000615
36  0.022979   0.000338
26  0.040424   0.008240
44 -0.016557   0.022780
12 -0.014613   0.008455
54 -0.031417   0.002610
3  -0.009695   0.000623


In [ ]:
# 查看模型的回归系数
coefficients = pd.DataFrame({'Variable': X_train.columns, 'Coefficient': model_cv.coef_})
print(coefficients)

            Variable  Coefficient
0              const     0.000000
1  ln(CPI_ratio_CHN)    -0.873930
2  ln(PPI_ratio_CHN)     0.022518
3   ln(UR_ratio_CHN)    -0.035290
4   ln(FR_ratio_CHN)     0.144990
5  ln(INT_ratio_CHN)    -0.012624
6   ln(IP_ratio_CHN)    -0.000761
7   ln(CA_ratio_CHN)    -0.001445


The OLS regression using all data yielded an R-squared of 19.7%, indicating that the model explains approximately 19.7% of the variation in the dependent variable (CNYUSD). The Prob(F-statistic) of 0.136 exceeds 0.05, suggesting that the overall model is not statistically significant, meaning the explanatory variables do not significantly explain changes in the dependent variable. While none of the coefficients are statistically significant, the p-values for CPI_ratio_CHN, FR_ratio_CHN, INT_ratio_CHN, and CA_ratio_CHN are close to significance. The negative coefficient for CPI implies that an increase in China's CPI relative to the U.S. could lead to RMB depreciation. The positive coefficient for foreign reserves (FR) suggests that higher reserves might result in RMB appreciation, while the negative coefficient for CA indicates that an increase in the current account balance between China and the U.S. could contribute to RMB depreciation. The model's limitations may stem from fundamental differences in U.S. and Chinese economic structures and government policies, as factors influencing exchange rates are highly complex and dynamic. Simple linear regression may not fully capture these nuances, suggesting that additional factors or adjustments to the model structure are required to improve explanatory power.

In [ ]:
import statsmodels.api as sm

# 假设你已经有自变量 X 和因变量 Y
# 你可能需要给 X 数据集添加一个常数项（截距项）
X_with_const = sm.add_constant(X)  # 添加截距项

# 使用 statsmodels 进行 OLS 回归
model = sm.OLS(Y, X_with_const)
results = model.fit()

# 查看回归结果，包括系数的显著性、p 值等
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 CNYUSD   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.080
Method:                 Least Squares   F-statistic:                     1.684
Date:                Fri, 25 Oct 2024   Prob (F-statistic):              0.136
Time:                        16:33:05   Log-Likelihood:                 136.38
No. Observations:                  56   AIC:                            -256.8
Df Residuals:                      48   BIC:                            -240.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0019      0.00